In [11]:
import pandas as pd
import numpy as np

In [12]:
import requests
import pdfplumber
import pandas as pd
from io import BytesIO

def extract_table_from_page_2(pdf_url):
    # Fetch the PDF file directly from the URL
    response = requests.get(pdf_url)
    
    if response.status_code == 200:
        # Load the PDF from bytes in memory
        pdf_data = BytesIO(response.content)
        
        with pdfplumber.open(pdf_data) as pdf:
            # Ensure the PDF has at least 2 pages
            if len(pdf.pages) >= 2:
                # Extract the table from page 2 (index 1 because it's zero-indexed)
                page = pdf.pages[1]
                tables = page.extract_tables()
                
                if tables:
                    # Assuming there's only one table on page 2
                    table = tables[0]
                    df = pd.DataFrame(table)
                    return df
                else:
                    return "No tables found on page 2."
            else:
                return "The PDF doesn't have 2 pages."
    else:
        raise Exception("Failed to fetch the PDF")

# Example usage:
pdf_url = "https://www.harti.gov.lk/images/download/market_information/2024/october/daily_21-10-2024.pdf"
df = extract_table_from_page_2(pdf_url)

if isinstance(df, pd.DataFrame):
    print("Table from page 2 as DataFrame:")
    print(df)
else:
    print(df)

Table from page 2 as DataFrame:
                         0                   1              2   \
0                   Variety          21/10/2024     21/10/2024   
1                      None  Peliyagoda\nMarket  Kandy\nMarket   
2      Up Country Vegetable                None                  
3                     Beans            360- 450       350- 380   
4                    Carrot            100- 120        90- 100   
5                     Leeks            140- 150       180- 200   
6                 Beet root             70- 120        90- 100   
7       Beet root (N Eliya)                   -       130- 140   
8                  Knolkhol             80- 100        90- 100   
9                   Raddish             60- 100         60- 70   
10        Cabbage (N'Eliya)                   -       120- 130   
11          Cabbage (Kandy)            100- 120        80- 100   
12                   Tomato            220- 280       180- 200   
13    Low country Vegetable                N

In [13]:
df.head(20)

,0,1,2,3,4,5,6,7,8,9,10
0,Variety,21/10/2024,21/10/2024,21/10/2024,20/10/2024,21/10/2024,21/10/2024,21/10/2024,21/10/2024,21/10/2024,20/10/2024
1,None,Peliyagoda\nMarket,Kandy\nMarket,Dambulla\nMarket,Meegoda\nMarket,Norochchole\nMarket,Thambuththegama\nMarket,Keppetipola\nMarket,Nuwaraeliya\nMarket,Bandarawela\nMarket,Veyangoda\nMarket
2,Up Country Vegetable,None,,,,,,,,,
3,Beans,360- 450,350- 380,300- 350,360- 460,-,400 - 450,380- 400,380 - 420,-,380- 430
4,Carrot,100- 120,90- 100,80- 90,100- 120,80- 150,130 - 140,80- 100,90 - 100,-,90- 120
5,Leeks,140- 150,180- 200,130- 150,170- 240,200- 250,200 - 250,150- 170,160 - 170,-,200- 240
6,Beet root,70- 120,90- 100,50- 70,90- 110,90- 110,100 - 120,-,-,-,90- 100
7,Beet root (N Eliya),-,130- 140,80- 90,,-,-,110- 120,120 - 140,-,-
8,Knolkhol,80- 100,90- 100,100- 120,120- 160,-,110 - 120,90- 100,120 - 130,-,110- 160
9,Raddish,60- 100,60- 70,30- 45,90- 110,40- 50,80 - 100,40- 50,70 - 90,-,80- 110


In [21]:
import re
from statistics import mean

def get_new_data(df):
    date = pd.to_datetime(df.iloc[0, 1]).date()
    col_Kandy = None
    col_Dambulla = None

    for col in range(len(df.columns)):
        if df.iloc[1, col] == "Kandy\nMarket":
            col_Kandy = col
        elif df.iloc[1, col] == "Dambulla\nMarket":
            col_Dambulla = col

    beans = []
    carrot = []
    leeks = []
    tomato = []

    for row in range(len(df)):
        if df.iloc[row, 0] == "Beans":
            beans.append(df.iloc[row, col_Kandy])
            beans.append(df.iloc[row, col_Dambulla])
        elif df.iloc[row, 0] == "Carrot":
            carrot.append(df.iloc[row, col_Kandy])
            carrot.append(df.iloc[row, col_Dambulla])
        elif df.iloc[row, 0] == "Leeks":
            leeks.append(df.iloc[row, col_Kandy])
            leeks.append(df.iloc[row, col_Dambulla])
        elif df.iloc[row, 0] == "Tomato":
            tomato.append(df.iloc[row, col_Kandy])
            tomato.append(df.iloc[row, col_Dambulla])

    beans = [round(mean([float(value) for value in re.findall(r'\d+\.?\d*', beanPrice)]), 2) for beanPrice in beans]
    
    
    Beans = pd.DataFrame({
        'Date': [date],
        'Kandy': [beans[0]],
        'Dambulla': [beans[1]]
    })

    Carrot = pd.DataFrame({
        'Date': [date],
        'Kandy': [carrot[0]],
        'Dambulla': [carrot[1]]
    })

    Leeks = pd.DataFrame({
        'Date': [date],
        'Kandy': [leeks[0]],
        'Dambulla': [leeks[1]]
    })

    Tomato = pd.DataFrame({
        'Date': [date],
        'Kandy': [tomato[0]],
        'Dambulla': [tomato[1]]
    })

    return Beans, Carrot, Leeks, Tomato

In [22]:
dataframes = get_new_data(df)
dataframes[0].head()

C:\Users\HP\AppData\Local\Temp\ipykernel_10916\893124233.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  date = pd.to_datetime(df.iloc[0, 1]).date()


,Date,Kandy,Dambulla
0,2024-10-21,365.0,325.0


In [1]:
import re
import pandas as pd
from statistics import mean
import firebase_admin
from firebase_admin import credentials, firestore
import streamlit as st
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import models # type: ignore
import pickle
import numpy as np
import os
import pdfplumber
import requests
from io import BytesIO
import re
from statistics import mean
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
vegetables = ['Beans', 'Carrot', 'Leeks', 'Tomato']
markets = ['Kandy', 'Dambulla']
date_range = pd.date_range(start='2023-12-29', end='2024-10-24')

In [11]:
defaultDf = pd.DataFrame(date_range, columns=['Date'])
newDataframes = {}

for market in markets:
    defaultDf[market] = 0

for vegetable in vegetables:
    newDataframes[vegetable] = defaultDf.copy()

st.dataframe(defaultDf)

for i, date in enumerate(date_range):
    try:
        print(f'starting date {date}')
        year = date.strftime('%Y')
        month = date.strftime('%B')
        dateStr = date.strftime('%d-%m-%Y')
        url = f'https://www.harti.gov.lk/images/download/market_information/{year}/{month.lower()}/daily_{dateStr}.pdf'
        print(f'fetching data from {url}')
        response = requests.get(url)

        if response.status_code != 200:
            url = f'https://www.harti.gov.lk/images/download/market_information/{year}/{month}/daily_{dateStr}.pdf'
            print(f'fetching data from {url}')
            response = requests.get(url)
        
        if response.status_code != 200:
            newDate = date.replace(year=date.year-1)
            dateStr = newDate.strftime('%d-%m-%Y')
            url = f'https://www.harti.gov.lk/images/download/market_information/{year}/daily/{month.lower()}/daily_{dateStr}.pdf'
            print(f'fetching data from {url}')
            response = requests.get(url)
        
        if response.status_code == 200:
            pdf_data = BytesIO(response.content)
            with pdfplumber.open(pdf_data) as pdf:
                if len(pdf.pages) == 2:
                    page = pdf.pages[0]
                elif len(pdf.pages) >= 2:
                    page = pdf.pages[1]
                
                tables = page.extract_tables()
                
                if tables:
                    table = tables[0]
                    df = pd.DataFrame(table)
                    print(df.head(10))
                    print(df.iloc[1, 1])

                    marketColumns = {}
                    for col in range(len(df.columns)):
                        for market in markets:
                            if df.iloc[1, col] == f'{market}':
                                marketColumns[market] = col
                            elif df.iloc[1, col] == f'{market}\nMarket':
                                marketColumns[market] = col

                    for row in range(len(df)):
                        for vegetable in vegetables:
                            if df.iloc[row, 0] == vegetable:
                                for market in markets:
                                    value = df.iloc[row, marketColumns[market]]
                                    value = round(mean([float(value) for value in re.findall(r'\d+\.?\d*', value)]), 2)
                                    newDataframes[vegetable].iloc[i, markets.index(market)+1] = value
                                    print(f'added value to {vegetable} in {market} on {date}')
                else:
                    continue
        else:
            print(f'failed to fetch data from {url}')
            continue

    except Exception as e:
        print(f"An error occurred: {e}")
        continue  # This ensures the loop will continue even after an error


2024-10-24 11:46:22.030 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-24 11:46:22.031 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


starting date 2023-12-29 00:00:00
fetching data from https://www.harti.gov.lk/images/download/market_information/2023/december/daily_29-12-2023.pdf
fetching data from https://www.harti.gov.lk/images/download/market_information/2023/December/daily_29-12-2023.pdf
fetching data from https://www.harti.gov.lk/images/download/market_information/2023/daily/december/daily_29-12-2022.pdf
                     0                   1              2                 3   \
0               Variety          29/12/2023     29/12/2023        29/12/2023   
1                  None  Peliyagoda\nMarket  Kandy\nMarket  Dambulla\nMarket   
2  Up Country Vegetable                None                                    
3                 Beans            450- 500       580- 600          450- 500   
4                Carrot            600- 650       600- 650          700- 750   
5                 Leeks            400- 420       390- 400          480- 550   
6             Beet root            380- 400       480- 500

C:\Users\HP\AppData\Local\Temp\ipykernel_14692\3596061113.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '57.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  newDataframes[vegetable].iloc[i, markets.index(market)+1] = value


fetching data from https://www.harti.gov.lk/images/download/market_information/2024/April/daily_05-04-2024.pdf
                   0                   1              2                 3   \
0             Variety          2024-04-05     2024-04-05        2024-04-05   
1                None  Peliyagoda\nMarket  Kandy\nMarket  Dambulla\nMarket   
2                                                                            
3               Beans             70- 100        80- 100            70- 90   
4              Carrot            180- 200       280- 300          220- 250   
5               Leeks            120- 150       160- 180          140- 180   
6           Beet root            150- 180        170 180          110- 130   
7  Beet Root(N'Eliya)                   -       180- 200          130- 140   
8            Knolkhol              40- 60         50- 70            50- 70   
9             Raddish              30- 50         20- 40            20- 25   

                4             

C:\Users\HP\AppData\Local\Temp\ipykernel_14692\3596061113.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '72.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  newDataframes[vegetable].iloc[i, markets.index(market)+1] = value


                     0                   1              2                 3   \
0               Variety          16/10/2024     16/10/2024        16/10/2024   
1                  None  Peliyagoda\nMarket  Kandy\nMarket  Dambulla\nMarket   
2  Up Country Vegetable                None                                    
3                 Beans            300- 400       300- 350          260- 350   
4                Carrot             60- 100         80- 90           70- 100   
5                 Leeks            150- 180       160- 180          100- 150   
6             Beet root             80- 100         80- 90            50- 70   
7   Beet root (N Eliya)                   -       100- 130           80- 100   
8              Knolkhol             80- 120         70- 80            70- 90   
9               Raddish              50- 80         60- 70            20- 40   

                4                    5                        6   \
0       15/10/2024           16/10/2024            

C:\Users\HP\AppData\Local\Temp\ipykernel_14692\3596061113.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '112.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  newDataframes[vegetable].iloc[i, markets.index(market)+1] = value


fetching data from https://www.harti.gov.lk/images/download/market_information/2024/October/daily_24-10-2024.pdf
fetching data from https://www.harti.gov.lk/images/download/market_information/2024/daily/october/daily_24-10-2023.pdf
failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/october/daily_24-10-2023.pdf


In [24]:
newDataframes[vegetables[1]]

,Date,Kandy,Dambulla
0,2023-12-29,625,725.0
1,2023-12-30,660,740.0
2,2023-12-31,615,675.0
3,2024-01-01,610,625.0
4,2024-01-02,660,600.0
...,...,...,...
296,2024-10-20,75,90.0
297,2024-10-21,95,85.0
298,2024-10-22,105,80.0
299,2024-10-23,85,77.5


In [20]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")

service = Service(executable_path="../testing/chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

defaultDf = pd.DataFrame(date_range, columns=['Date'])

try:
    driver.get('https://www.cbsl.gov.lk/en/rates-and-indicators/exchange-rates/daily-buy-and-sell-exchange-rates')

    iframe_element = driver.find_element(By.XPATH, "//iframe[@src='/cbsl_custom/exratestt/exratestt.php']")
    driver.switch_to.frame(iframe_element)

    submit_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, "submit_button"))
    )
    submit_button.click()

    dates = len(date_range)
    if dates == 1:
        timeframe = None
    elif dates <= 7:
        timeframe = "1week"
    elif dates <= 14:
        timeframe = "2weeks"
    elif dates <= 30:
        timeframe = "1month"
    elif dates <= 90:
        timeframe = "3months"
    elif dates <= 180:
        timeframe = "6months"
    else:
        timeframe = "1year"

    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//button[@onclick='setDateValues_{timeframe}();']"))
    )
    time_button.click()

    usd_table_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//h2[text()=' United States Dollar ']"))
    )

    usd_table_html = usd_table_element.get_attribute('outerHTML')

    soup = BeautifulSoup(usd_table_html, 'html.parser')
    usd_table = pd.read_html(str(soup))[0]
    usd_table = usd_table[['Date', 'Buy Rate (LKR)']]
    usd_table.rename(columns={'Buy Rate (LKR)': 'Buy Rate'}, inplace=True)
    usd_table = usd_table.round(2)
    usd_table['Date'] = pd.to_datetime(usd_table['Date']).dt.date
    usd_table = usd_table[(usd_table['Date'] >= date_range[0]) & (usd_table['Date'] <= date_range[-1])]
finally:
    driver.quit()
usd_table = usd_table.set_index('Date').reindex(date_range).reset_index()
usd_table.rename(columns={'index': 'Date'}, inplace=True)
usd_table['Buy Rate'] = usd_table['Buy Rate'].fillna(0)

C:\Users\HP\AppData\Local\Temp\ipykernel_14692\2984153518.py:50: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  usd_table = pd.read_html(str(soup))[0]


ImportError: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.

In [35]:
tempdf = {
    'col1': [1, 2, 3, 4, 5],
    'col2': [6, 7, 8, 9, 10],
    'col3': [11, 12, 13, 14, 15]
}

tempdf = pd.DataFrame(tempdf)

tempdf.iloc[1, 1] = 0
tempdf.iloc[2, 1] = 0
tempdf.iloc[4, 1] = 0
tempdf = tempdf.replace(0, np.nan)
tempdf = tempdf.interpolate(type='linear', limit_direction='both')
tempdf.head()

,col1,col2,col3
0,1,6.0,11
1,2,7.0,12
2,3,8.0,13
3,4,9.0,14
4,5,9.0,15


In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
import streamlit as st
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import models # type: ignore
import pickle
import numpy as np
import os
import pdfplumber
import requests
from io import BytesIO
import re
from statistics import mean
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup

vegetables = ['Beans', 'Carrot', 'Leeks', 'Tomato']
markets = ['Kandy', 'Dambulla']

_date_range = pd.date_range(start='2023-12-29', end='2024-10-24')

def getNewVegData(date_range, vegetables, markets):

    defaultDf = pd.DataFrame(date_range, columns=['Date'])
    newDataframes = {}

    for market in markets:
        defaultDf[market] = 0

    for vegetable in vegetables:
        newDataframes[vegetable] = defaultDf.copy()

    for i, date in enumerate(date_range):
        try:
            year = date.strftime('%Y')
            month = date.strftime('%B')
            dateStr = date.strftime('%d-%m-%Y')
            url = f'https://www.harti.gov.lk/images/download/market_information/{year}/{month.lower()}/daily_{dateStr}.pdf'
            response = requests.get(url)

            if response.status_code != 200:
                url = f'https://www.harti.gov.lk/images/download/market_information/{year}/{month}/daily_{dateStr}.pdf'
                response = requests.get(url)
            
            if response.status_code != 200:
                newDate = date.replace(year=date.year-1)
                dateStr = newDate.strftime('%d-%m-%Y')
                url = f'https://www.harti.gov.lk/images/download/market_information/{year}/daily/{month.lower()}/daily_{dateStr}.pdf'
                response = requests.get(url)
            
            if response.status_code == 200:
                pdf_data = BytesIO(response.content)
                with pdfplumber.open(pdf_data) as pdf:
                    if len(pdf.pages) == 2:
                        page = pdf.pages[0]
                    elif len(pdf.pages) >= 2:
                        page = pdf.pages[1]
                    
                    tables = page.extract_tables()
                    
                    if tables:
                        table = tables[0]
                        df = pd.DataFrame(table)

                        marketColumns = {}
                        for col in range(len(df.columns)):
                            for market in markets:
                                if df.iloc[1, col] == f'{market}':
                                    marketColumns[market] = col
                                elif df.iloc[1, col] == f'{market}\nMarket':
                                    marketColumns[market] = col

                        for row in range(len(df)):
                            for vegetable in vegetables:
                                if df.iloc[row, 0] == vegetable:
                                    for market in markets:
                                        value = df.iloc[row, marketColumns[market]]
                                        value = round(mean([float(value) for value in re.findall(r'\d+\.?\d*', value)]), 2)
                                        newDataframes[vegetable].iloc[i, markets.index(market)+1] = value
                    else:
                        continue
            else:
                print(f'failed to fetch data from {url}')
                continue

        except Exception as e:
            print(f"An error occurred: {e}")
            continue  # This ensures the loop will continue even after an error

    return newDataframes

def getNewBuyRate(date_range):
    # Set up headless Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode (without GUI)
    chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration (optional)
    chrome_options.add_argument("--window-size=1920,1080")  # Optional, to ensure the page has enough space

    # Initialize the WebDriver with headless mode
    service = Service(executable_path="../chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=chrome_options)

    date_range = pd.date_range(start='2023-12-29', end='2024-10-24')
    defaultDf = pd.DataFrame(date_range, columns=['Date'])
    defaultDf['Buy Rate'] = 0

    try:
        # Open the target URL
        driver.get('https://www.cbsl.gov.lk/en/rates-and-indicators/exchange-rates/daily-buy-and-sell-exchange-rates')

        # Switch to the iframe
        iframe_element = driver.find_element(By.XPATH, "//iframe[@src='/cbsl_custom/exratestt/exratestt.php']")
        driver.switch_to.frame(iframe_element)

        # Wait for the first submit button to be clickable
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, "submit_button"))
        )
        submit_button.click()

        dates = len(date_range)
        if dates == 1:
            timeframe = None
        elif dates <= 7:
            timeframe = "1week"
        elif dates <= 14:
            timeframe = "2weeks"
        elif dates <= 30:
            timeframe = "1month"
        elif dates <= 90:
            timeframe = "3months"
        elif dates <= 180:
            timeframe = "6months"
        else:
            timeframe = "1year"

        # Wait for the new layout to load and the corresponding button to be clickable
        one_week_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//button[@onclick='setDateValues_{timeframe}();']"))
        )
        one_week_button.click()

        # Locate the "United States Dollar" section by its <h2> tag
        h2_usd = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//h2[text()=' United States Dollar ']"))
        )

        # Find the table following the <h2> tag
        usd_table_element = h2_usd.find_element(By.XPATH, "following-sibling::div[@class='table-responsive']//table")

        # Extract the table's HTML
        usd_table_html = usd_table_element.get_attribute('outerHTML')

        # Parse the table with BeautifulSoup and convert it to a pandas DataFrame
        soup = BeautifulSoup(usd_table_html, 'html.parser')
        usd_table = pd.read_html(str(soup))[0]
        usd_table = usd_table[['Date', 'Buy Rate (LKR)']]
        usd_table.rename(columns={'Buy Rate (LKR)': 'Buy Rate'}, inplace=True)
        usd_table = usd_table.round(2)
        usd_table['Date'] = pd.to_datetime(usd_table['Date']).dt.date
        usd_table = usd_table[(usd_table['Date'] >= date_range[0].date()) & (usd_table['Date'] <= date_range[-1].date())]
        usd_table = usd_table.set_index('Date').reindex(date_range).reset_index()
        usd_table.rename(columns={'index': 'Date'}, inplace=True)
        usd_table['Buy Rate'] = usd_table['Buy Rate'].fillna(0)

    finally:
        # Quit the WebDriver
        driver.quit()
    
    if usd_table is not None:
        print(usd_table)
        return usd_table
    else:
        print(defaultDf)
        return defaultDf

newBuyRate = getNewBuyRate(_date_range)
newVegData = getNewVegData(_date_range, vegetables, markets)
newData = {}

for veg, df in newVegData.items():
    newData[veg] = pd.merge(df, newBuyRate, on='Date', how='left')
    newData[veg] = newData[veg].replace(0, np.nan)
    newData[veg] = newData[veg].interpolate(method='linear', limit_direction='both')

for veg, df in newData.items():
    print(f"### {veg}")
    print(df)

C:\Users\HP\AppData\Local\Temp\ipykernel_9028\218611626.py:164: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  usd_table = pd.read_html(str(soup))[0]


          Date  Buy Rate
0   2023-12-29    319.18
1   2023-12-30      0.00
2   2023-12-31      0.00
3   2024-01-01    319.23
4   2024-01-02    316.93
..         ...       ...
296 2024-10-20      0.00
297 2024-10-21    288.61
298 2024-10-22    288.66
299 2024-10-23    288.63
300 2024-10-24    289.00

[301 rows x 2 columns]
failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/january/daily_25-01-2023.pdf
An error occurred: 'Kandy'
failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/february/daily_23-02-2023.pdf
An error occurred: mean requires at least one data point
An error occurred: 'Kandy'
An error occurred: 'Kandy'
An error occurred: 'Kandy'
An error occurred: 'Kandy'
An error occurred: 'Kandy'
An error occurred: 'Kandy'
An error occurred: 'Kandy'
An error occurred: 'Kandy'
An error occurred: 'Kandy'
An error occurred: 'Kandy'
An error occurred: 'Kandy'
failed to fetch data from https://www.hart

C:\Users\HP\AppData\Local\Temp\ipykernel_9028\218611626.py:87: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '57.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  newDataframes[vegetable].iloc[i, markets.index(market)+1] = value


failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/april/daily_12-04-2023.pdf
failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/april/daily_13-04-2023.pdf
failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/april/daily_14-04-2023.pdf
failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/april/daily_15-04-2023.pdf
failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/april/daily_23-04-2023.pdf
An error occurred: mean requires at least one data point
failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/may/daily_23-05-2023.pdf
failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/may/daily_24-05-2023.pdf
failed to fetch data from https://www.harti.gov.lk/images/download/market_infor

C:\Users\HP\AppData\Local\Temp\ipykernel_9028\218611626.py:87: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '72.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  newDataframes[vegetable].iloc[i, markets.index(market)+1] = value


failed to fetch data from https://www.harti.gov.lk/images/download/market_information/2024/daily/october/daily_17-10-2023.pdf


C:\Users\HP\AppData\Local\Temp\ipykernel_9028\218611626.py:87: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '112.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  newDataframes[vegetable].iloc[i, markets.index(market)+1] = value


### Beans
          Date  Kandy  Dambulla    Buy Rate
0   2023-12-29  590.0     475.0  319.180000
1   2023-12-30  575.0     425.0  319.196667
2   2023-12-31  575.0     425.0  319.213333
3   2024-01-01  425.0     325.0  319.230000
4   2024-01-02  575.0     440.0  316.930000
..         ...    ...       ...         ...
296 2024-10-20  365.0     375.0  288.583333
297 2024-10-21  365.0     325.0  288.610000
298 2024-10-22  275.0     315.0  288.660000
299 2024-10-23  235.0     225.0  288.630000
300 2024-10-24  210.0     215.0  289.000000

[301 rows x 4 columns]
### Carrot
          Date  Kandy  Dambulla    Buy Rate
0   2023-12-29  625.0     725.0  319.180000
1   2023-12-30  660.0     740.0  319.196667
2   2023-12-31  615.0     675.0  319.213333
3   2024-01-01  610.0     625.0  319.230000
4   2024-01-02  660.0     600.0  316.930000
..         ...    ...       ...         ...
296 2024-10-20   75.0      90.0  288.583333
297 2024-10-21   95.0      85.0  288.610000
298 2024-10-22  105.0      80.0

In [2]:
for veg, df in newData.items():
    df = df.round(2)
    print(f"### {veg}")
    print(df)

### Beans
          Date  Kandy  Dambulla  Buy Rate
0   2023-12-29  590.0     475.0    319.18
1   2023-12-30  575.0     425.0    319.20
2   2023-12-31  575.0     425.0    319.21
3   2024-01-01  425.0     325.0    319.23
4   2024-01-02  575.0     440.0    316.93
..         ...    ...       ...       ...
296 2024-10-20  365.0     375.0    288.58
297 2024-10-21  365.0     325.0    288.61
298 2024-10-22  275.0     315.0    288.66
299 2024-10-23  235.0     225.0    288.63
300 2024-10-24  210.0     215.0    289.00

[301 rows x 4 columns]
### Carrot
          Date  Kandy  Dambulla  Buy Rate
0   2023-12-29  625.0     725.0    319.18
1   2023-12-30  660.0     740.0    319.20
2   2023-12-31  615.0     675.0    319.21
3   2024-01-01  610.0     625.0    319.23
4   2024-01-02  660.0     600.0    316.93
..         ...    ...       ...       ...
296 2024-10-20   75.0      90.0    288.58
297 2024-10-21   95.0      85.0    288.61
298 2024-10-22  105.0      80.0    288.66
299 2024-10-23   85.0      77.5

In [13]:
githubData = {}
for veg in vegetables:
    githubData[veg] = pd.read_csv(f'../data/NewData/{veg}.csv')

for veg, df in githubData.items():
    print(f"### {veg}")
    print(df)

### Beans
            Date   Kandy  Dambulla  Buy Rate
0     2020-01-01  310.00    315.00    181.50
1     2020-01-02  274.17    279.17    181.03
2     2020-01-03  238.33    243.33    181.37
3     2020-01-04  202.50    207.50    181.35
4     2020-01-05  166.67    171.67    181.34
...          ...     ...       ...       ...
1091  2022-12-27  565.00    425.00    363.10
1092  2022-12-28  450.00    325.00    363.13
1093  2022-12-29  510.00    425.00    363.13
1094  2022-12-30  550.00    475.00    363.11
1095  2022-12-31  581.67    500.00    363.11

[1096 rows x 4 columns]
### Carrot
            Date   Kandy  Dambulla  Buy Rate
0     2020-01-01  296.43     305.0    181.50
1     2020-01-02  267.86     272.5    181.03
2     2020-01-03  239.29     240.0    181.37
3     2020-01-04  210.71     207.5    181.35
4     2020-01-05  182.14     175.0    181.34
...          ...     ...       ...       ...
1091  2022-12-27  245.00     200.0    363.10
1092  2022-12-28  210.00     200.0    363.13
1093  202

In [16]:
if not firebase_admin._apps:

    firebase_credentials = '../firebase.json'

    cred = credentials.Certificate(firebase_credentials)
    firebase_admin.initialize_app(cred)
db = firestore.client()

docs = db.collection('NewData').stream()
fbData = pd.DataFrame([doc.to_dict() for doc in docs]) # data stored in firebase

firebaseData = {}
for vegetable in vegetables:
    markets = [column for column in fbData.columns if vegetable in column]
    firebaseTempData = fbData[['Date'] + markets].copy()
    firebaseTempData['Date'] = pd.to_datetime(firebaseTempData['Date'], format='%d %B %Y at %H:%M:%S %Z%z', errors='coerce', utc=True).dt.date
    for market in markets:
        firebaseTempData.rename(columns={market: market.split('_')[0]}, inplace=True)
    firebaseData[vegetable] = firebaseTempData

In [17]:
for veg, df in firebaseData.items():
    print(f"### {veg}")
    print(df)

### Beans
           Date  Buy Rate   Kandy  Dambulla
0    2023-01-01    363.11  613.33     525.0
1    2023-01-02    363.11  645.00     550.0
2    2023-01-03    363.11  635.00     575.0
3    2023-01-04    363.11  525.00     565.0
4    2023-01-05    363.11  500.00     525.0
..          ...       ...     ...       ...
358  2023-12-25    325.99  555.00     466.0
359  2023-12-26    325.86  565.00     483.0
360  2023-12-27    325.74  575.00     500.0
361  2023-12-28    324.23  525.00     550.0
362  2023-12-29    323.92  590.00     475.0

[363 rows x 4 columns]
### Carrot
           Date  Dambulla  Buy Rate   Kandy
0    2023-01-01    130.00    363.11  138.33
1    2023-01-02     95.00    363.11   95.00
2    2023-01-03    100.00    363.11  105.00
3    2023-01-04    168.33    363.11  183.33
4    2023-01-05    236.67    363.11  261.67
..          ...       ...       ...     ...
358  2023-12-25    496.00    325.99  499.00
359  2023-12-26    523.00    325.86  532.00
360  2023-12-27    550.00    32

In [19]:
col_order = ['Date', 'Kandy', 'Dambulla', 'Buy Rate']

for veg, df in githubData.items():
    df = df.reindex(columns=col_order)
    print(f"### {veg}")
    print(df)

### Beans
            Date   Kandy  Dambulla  Buy Rate
0     2020-01-01  310.00    315.00    181.50
1     2020-01-02  274.17    279.17    181.03
2     2020-01-03  238.33    243.33    181.37
3     2020-01-04  202.50    207.50    181.35
4     2020-01-05  166.67    171.67    181.34
...          ...     ...       ...       ...
1091  2022-12-27  565.00    425.00    363.10
1092  2022-12-28  450.00    325.00    363.13
1093  2022-12-29  510.00    425.00    363.13
1094  2022-12-30  550.00    475.00    363.11
1095  2022-12-31  581.67    500.00    363.11

[1096 rows x 4 columns]
### Carrot
            Date   Kandy  Dambulla  Buy Rate
0     2020-01-01  296.43     305.0    181.50
1     2020-01-02  267.86     272.5    181.03
2     2020-01-03  239.29     240.0    181.37
3     2020-01-04  210.71     207.5    181.35
4     2020-01-05  182.14     175.0    181.34
...          ...     ...       ...       ...
1091  2022-12-27  245.00     200.0    363.10
1092  2022-12-28  210.00     200.0    363.13
1093  202

In [28]:
dataframes = {}

for veg in vegetables:
    newData[veg]['Date'] = pd.to_datetime(newData[veg]['Date']).dt.date
    newData[veg] = newData[veg].reindex(columns=col_order)
    newData[veg] = newData[veg].round(2)
    githubData[veg]['Date'] = pd.to_datetime(githubData[veg]['Date']).dt.date
    githubData[veg] = githubData[veg].reindex(columns=col_order)
    githubData[veg] = githubData[veg].round(2)
    firebaseData[veg]['Date'] = pd.to_datetime(firebaseData[veg]['Date']).dt.date
    firebaseData[veg] = firebaseData[veg].reindex(columns=col_order)
    firebaseData[veg] = firebaseData[veg].round(2)
    dataframes[veg] = pd.concat([newData[veg], githubData[veg], firebaseData[veg]])
    dataframes[veg] = dataframes[veg].sort_values(by='Date').reset_index(drop=True)
    dataframes[veg] = dataframes[veg].drop_duplicates(subset='Date', keep='last')
    print(f"### {veg}")
    print(dataframes[veg])

### Beans
            Date   Kandy  Dambulla  Buy Rate
0     2020-01-01  310.00    315.00    181.50
1     2020-01-02  274.17    279.17    181.03
2     2020-01-03  238.33    243.33    181.37
3     2020-01-04  202.50    207.50    181.35
4     2020-01-05  166.67    171.67    181.34
...          ...     ...       ...       ...
1755  2024-10-20  365.00    375.00    288.58
1756  2024-10-21  365.00    325.00    288.61
1757  2024-10-22  275.00    315.00    288.66
1758  2024-10-23  235.00    225.00    288.63
1759  2024-10-24  210.00    215.00    289.00

[1759 rows x 4 columns]
### Carrot
            Date   Kandy  Dambulla  Buy Rate
0     2020-01-01  296.43     305.0    181.50
1     2020-01-02  267.86     272.5    181.03
2     2020-01-03  239.29     240.0    181.37
3     2020-01-04  210.71     207.5    181.35
4     2020-01-05  182.14     175.0    181.34
...          ...     ...       ...       ...
1755  2024-10-20   75.00      90.0    288.58
1756  2024-10-21   95.00      85.0    288.61
1757  202

In [29]:
splitDate = pd.to_datetime('2024-01-01').date()
print(splitDate)

2024-01-01


In [47]:
beforeData = {}
afterData = {}

for veg, df in dataframes.items():
    beforeData[veg] = df[df['Date'] < splitDate].reset_index(drop=True)
    afterData[veg] = df[df['Date'] >= splitDate].reset_index(drop=True)
    print(f"### {veg}")
    print(beforeData[veg].tail())
    print(afterData[veg].tail())

### Beans
            Date  Kandy  Dambulla  Buy Rate
1456  2023-12-27  575.0     500.0    325.74
1457  2023-12-28  525.0     550.0    324.23
1458  2023-12-29  590.0     475.0    323.92
1459  2023-12-30  575.0     425.0    319.20
1460  2023-12-31  575.0     425.0    319.21
           Date  Kandy  Dambulla  Buy Rate
293  2024-10-20  365.0     375.0    288.58
294  2024-10-21  365.0     325.0    288.61
295  2024-10-22  275.0     315.0    288.66
296  2024-10-23  235.0     225.0    288.63
297  2024-10-24  210.0     215.0    289.00
### Carrot
            Date  Kandy  Dambulla  Buy Rate
1456  2023-12-27  565.0     550.0    325.74
1457  2023-12-28  565.0     575.0    324.23
1458  2023-12-29  625.0     725.0    323.92
1459  2023-12-30  660.0     740.0    319.20
1460  2023-12-31  615.0     675.0    319.21
           Date  Kandy  Dambulla  Buy Rate
293  2024-10-20   75.0      90.0    288.58
294  2024-10-21   95.0      85.0    288.61
295  2024-10-22  105.0      80.0    288.66
296  2024-10-23   85.

In [35]:
for veg, df in beforeData.items():
    df.to_csv(f'../data/{veg}.csv', index=False)

In [34]:
import firebase_admin
from firebase_admin import credentials, firestore

In [48]:
for veg, df in afterData.items():
    df.set_index('Date', inplace=True)
    for col in df.columns:
        df.rename(columns={col: f'{col}_{veg}'}, inplace=True)
        afterData[veg] = df

In [56]:
combined_df = pd.concat(afterData.values(), axis=1)

combined_df = combined_df.reset_index()
combined_df['Date'] = pd.to_datetime(combined_df['Date'])

combined_df.tail()

,Date,Kandy_Beans,Dambulla_Beans,Buy Rate_Beans,Kandy_Carrot,Dambulla_Carrot,Buy Rate_Carrot,Kandy_Leeks,Dambulla_Leeks,Buy Rate_Leeks,Kandy_Tomato,Dambulla_Tomato,Buy Rate_Tomato
293,2024-10-20,365.0,375.0,288.58,75.0,90.0,288.58,190.0,235.0,288.58,190.0,190.0,288.58
294,2024-10-21,365.0,325.0,288.61,95.0,85.0,288.61,190.0,140.0,288.61,190.0,210.0,288.61
295,2024-10-22,275.0,315.0,288.66,105.0,80.0,288.66,160.0,120.0,288.66,165.0,220.0,288.66
296,2024-10-23,235.0,225.0,288.63,85.0,77.5,288.63,155.0,112.5,288.63,190.0,180.0,288.63
297,2024-10-24,210.0,215.0,289.00,85.0,75.0,289.00,130.0,160.0,289.00,195.0,200.0,289.00


In [57]:
if not firebase_admin._apps:
    cred = credentials.Certificate('./firebase.json')
    firebase_admin.initialize_app(cred)

db = firestore.client()

for _, row in combined_df.iterrows():
    
    doc_id = str(row['Date'].date())
    row_dict = row.to_dict()

    db.collection('Data').document(doc_id).set(row_dict)